## 資料前處理

In [ ]:
#@title 載入資料

%%capture

import re
from datetime import timedelta
import numpy as np

from matplotlib.font_manager import FontProperties as font
import matplotlib.font_manager as fm
!mkdir "fonts"
!wget -O "fonts/TaipeiSansTCBeta-Regular.ttf" https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
tcfont = font(fname="/content/fonts/TaipeiSansTCBeta-Regular.ttf")
fm.fontManager.addfont("/content/fonts/TaipeiSansTCBeta-Regular.ttf")


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


def str_to_time(time_str):
  if time_str == '': return 0
  match = re.match(r"(\d+)\s*時\s*(\d+)\s*分\s*(\d+)\s*秒", time_str)

  if match:
    hours = int(match.group(1))
    minutes = int(match.group(2))
    seconds = int(match.group(3))

    # Create a datetime.time object
    time_obj = timedelta(hours=hours, minutes=minutes, seconds=seconds).total_seconds()
    return time_obj
  else:
    print(f"The input string '{time_str}' is not in the expected format.")

def str_to_null(str):
  if str == "Null":
    return np.nan
  if str == "":
    return np.nan
  return str

def df_str_to_time(df):
  # df.drop('stuid', axis=1, inplace=True)

  df['在線時間'] = df['在線時間'].apply(str_to_time)
  df['本月素材總使用時間'] = df['本月素材總使用時間'].apply(str_to_time)

  df['本月瀏覽影片時間'] = df['本月瀏覽影片時間'].apply(str_to_time)
  df['影片加總時間'] = df['影片加總時間'].apply(str_to_time)

  df['本月測驗時間'] = df['本月測驗時間'].apply(str_to_time)
  df['測驗加總時間'] = df['測驗加總時間'].apply(str_to_time)

  df['本月語音作業時間'] = df['本月語音作業時間'].apply(str_to_time)
  df['語音作業加總時間'] = df['語音作業加總時間'].apply(str_to_time)

  df['本月音檔時間'] = df['本月音檔時間'].apply(str_to_time)
  df['音檔加總時間'] = df['音檔加總時間'].apply(str_to_time)

  df['stuid'] = df['stuid'].apply(str_to_null)
  df['stuid'] = df['stuid'].astype(int, errors='ignore')

  df['stuid'] = df['stuid'].fillna(0).astype('int64')

  return df

import pandas as pd

# Students
!gdown --id 18XHRJBKxPc9Mxgx11tPbDY_YD0oORe-C
dfstu = pd.read_csv('lt_stu.csv', skiprows=[])

# 2024/02
gs2402 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1CQFJo0vGujmGS3jOn3EtmfuAiI4B2BaQpvxt2crFDY0/edit')
s2402 = gs2402.worksheet('Sheet1').get_all_values()
df2402 = df_str_to_time(pd.DataFrame(s2402[1:], columns=s2402[0]))

# 2024/03
gs2403 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1CPWcr8GTZ4kLyCUUbLL7A8c69hPT03eSpDtKSfBDCuI/edit')
s2403 = gs2403.worksheet('Sheet1').get_all_values()
df2403 = df_str_to_time(pd.DataFrame(s2403[1:], columns=s2403[0]))

# 2024/04
gs2404 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1irJehRvsMaWOd0TM2bn6bgxs6i2KNoNpuB8qWulxXgk/edit')
s2404 = gs2404.worksheet('Sheet1').get_all_values()
df2404 = df_str_to_time(pd.DataFrame(s2404[1:], columns=s2404[0]))

# 2024/05
gs2405 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1chP2EbPmY5WkAka50M4hYKWjuhM3mM7T6DuKbK_hsm8/edit')
s2405 = gs2405.worksheet('Sheet1').get_all_values()
df2405 = df_str_to_time(pd.DataFrame(s2405[1:], columns=s2405[0]))

# 2024/06
gs2406 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hYylelU_571nfX05etK8flxUQEE6OkSiKdlcaNqjY3U/edit')
s2406 = gs2406.worksheet('Sheet1').get_all_values()
df2406 = df_str_to_time(pd.DataFrame(s2406[1:], columns=s2406[0]))

In [ ]:
# @title 合併學生資訊
# Concatenate the dataframes
df_2402_2406 = pd.concat([df2402, df2403, df2404, df2405, df2406], ignore_index=True)

dfstu_unique = dfstu.drop_duplicates(subset='stuid', keep='first')
dfstuid = dfstu_unique[["stuid", "sex", "birth"]]

df_2402_2406_stu = pd.merge(dfstuid, df_2402_2406, on='stuid', how='inner')
# df_2402_2406_stu.dropna(inplace=True)
df_2402_2406_stu['sex'] = df_2402_2406_stu['sex'].astype(int)
# Step 1: Define the mapping dictionary
sex_mapping = {0: "生理男", 1: "生理女"}
# Step 2: Replace the values in the 'sex' column
df_2402_2406_stu['sex'] = df_2402_2406_stu['sex'].replace(sex_mapping)


# 新增年級欄位

from datetime import datetime
# 設定當前年份
current_year = datetime.now().year
# 將 "生日" 欄位轉換為 datetime 格式
df_2402_2406_stu['birth'] = pd.to_datetime(df_2402_2406_stu['birth'], format='mixed')

# 設定年級的起始和結束日期
grade_start_date = {
    1: '2016-09-02',  # 一年級從每年的 9 月 1 日開始
    2: '2015-09-02',
    3: '2014-09-02',
    4: '2013-09-02',
    5: '2012-09-02',
    6: '2011-09-02'
}

grade_end_date = {
    1: '2017-09-01',  # 一年級到次年的 8 月 31 日結束
    2: '2016-09-01',
    3: '2015-09-01',
    4: '2014-09-01',
    5: '2013-09-01',
    6: '2012-09-01'
}

# 計算年級
def calculate_grade(birth_date):
    for grade in range(1, 7):  # 假設只計算小學六年級內的學生
        start_date = datetime.strptime(f"{grade_start_date[grade]}", "%Y-%m-%d")
        end_date = datetime.strptime(f"{grade_end_date[grade]}", "%Y-%m-%d")
        if start_date <= birth_date < end_date:
            return grade
    return 0  # 超過範圍或未入學的情況

def categorize_subject(course_name):
    # 定義各類別的關鍵字
    categories = {
        '國文': ['國語文教材', '學習扶助教材', '康軒國語', '二仁國語學扶', '國語日報', '成語故事',
                 '唐詩朗誦大挑戰', '國語四下', '翰林版─國小國語', '蔣老師國語課', 'L10追夢的翅膀',
                 'L11祝賀你，孩子', 'L1過故人莊', 'L2把愛傳下去', 'L5馬達加斯加', '寒溪二忠國語文寒假作業',
                 '少年噶瑪蘭', '南一版─國小國語', '國語', '國文', '文豪', '寫作', '唐詩', '成語', '國L', '青春第二課', '我所知道的康橋',
                 '國第9課', '第十課', '辦桌', '第八課', '油條報紙', 'L8', 'L6', '第九課', '第6課', '第十三課', 'L3', 'L9', '第三課',
                 '第四課', '第一課'],
        '閩語': ['真平版─國小閩語', '閩南語金安版五下', '閩', '本土語', '台語', '做伙來', '閒話講等路'],
        '英語': ['英文字母', '英文初級閱讀', '博幼英文', 'Dino', 'Fun World', 'What are you doing?',
                 '英文_HWG', '英文_WW', '暑假英語閱讀', '學習吧', '溫弟老師', '英語', 'English', '英文', '字母', 'Super Fun', 'Fish and Chips',
                 'G3', 'G4', 'G5', 'U1', 'U2', 'U3', 'U4', 'U5', 'U6', 'Grade', 'World', 'The Stone Garden', 'Follow Me', 'animals',
                 'Where are you'],
        '數學': ['翰林版─國小數學', '數學基礎與提升', '數學練習', '數學', '數', '柱體', '三角形', '乘法'],
        '社會': ['社會課程', '環境保護', '文化資產', '世界發生', '翰林版─國小社會', '社會', '歷史', '8歷', '地理', '公民', '8公',
                 '永續世界地球村'],
        '自然': ['生物與環境', '水保小教室', '南一版─國小自然', '自然與環境', '動物檢定', '翰林自然', '自', '速率', '電池', '機械', '生物'],
        '資訊': ['APP', '我阿嬤也會', '剪映', '程式', 'AI', '科技', '資訊', '錄音', '電腦']
    }

    # 檢查課程名稱並返回對應類別
    for category, keywords in categories.items():
        if any(keyword in course_name for keyword in keywords):
            return category
    return '其他'  # 若無匹配，歸類為「其他」

df_2402_2406_stu['grade'] = df_2402_2406_stu['birth'].apply(calculate_grade)
df_2402_2406_stu['courseCategory'] = df_2402_2406_stu['課程名稱'].apply(categorize_subject)

# df_2402_2406_stu = df_2402_2406_stu.drop_duplicates(subset=['stuid', 'month'])

# Update column name from "本月測驗時間" to "測驗時間"
df_2402_2406_stu.rename(columns={
    '本月素材總使用時間': '素材總使用時間',
    '本月瀏覽影片時間': '瀏覽影片時間',
    '本月測驗時間': '測驗時間',
    '本月語音作業時間': '語音作業時間',
    '本月音檔時間': '音檔時間',
    'sex': 'gender',
}, inplace=True)
df_2402_2406_stu.drop(['本月書籍瀏覽時間', '書籍加總時間', '本月課間教材時間', '課間教材加總時間'], axis=1, inplace=True)

df_2402_2406_stu.to_csv('df_2402_2406_stu.csv', index=False, encoding='utf-8')
df_2402_2406_stu

,stuid,gender,birth,在線時間,素材總使用時間,課程名稱,瀏覽影片時間,影片加總時間,測驗時間,測驗加總時間,語音作業時間,語音作業加總時間,音檔時間,音檔加總時間,grade,courseCategory
0,164811,生理女,1970-01-01 00:00:00.020090701,3376.0,100.0,901英語文,0.0,0.0,0.0,0.0,99.0,99.0,1.0,1.0,0,英語
1,164811,生理女,1970-01-01 00:00:00.020090701,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,其他
2,164811,生理女,1970-01-01 00:00:00.020090701,361.0,101.0,901英語文,0.0,0.0,0.0,0.0,101.0,101.0,0.0,0.0,0,英語
3,164811,生理女,1970-01-01 00:00:00.020090701,1144.0,34.0,901英語文,0.0,0.0,0.0,0.0,34.0,34.0,0.0,0.0,0,英語
4,136012,生理女,1970-01-01 00:00:00.020080923,1042.0,118.0,112-九仁英語課,0.0,0.0,0.0,0.0,118.0,118.0,0.0,0.0,0,英語
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21593,186607,生理女,1970-01-01 00:00:00.020121113,975.0,133.0,5下國語,0.0,0.0,0.0,0.0,133.0,133.0,0.0,0.0,0,國文
21594,186612,生理男,1970-01-01 00:00:00.020160331,474.0,172.0,閱讀寫作齊步走【小壁虎老師】,1.0,1.0,171.0,171.0,0.0,0.0,0.0,0.0,0,國文
21595,186752,生理男,1970-01-01 00:00:00.020170608,27.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,其他
21596,186845,生理女,1970-01-01 00:00:00.020140428,1437.0,472.0,四年級圖書館課,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,其他


In [ ]:
import pandas as pd
data = pd.read_csv("./df_2402_2406_stu.csv")
data = data[data['grade'] != 0]
data

,stuid,gender,birth,在線時間,素材總使用時間,課程名稱,瀏覽影片時間,影片加總時間,測驗時間,測驗加總時間,語音作業時間,語音作業加總時間,音檔時間,音檔加總時間,grade,courseCategory
1793,116930,生理男,2011-10-05 00:00:00.000000000,1060.0,516.0,學習吧－節慶英文故事館,511.0,511.0,0.0,0.0,0.0,0.0,0.0,0.0,6,英語
1794,116939,生理男,2011-09-10 00:00:00.000000000,935.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,其他
1795,116947,生理女,2012-05-05 00:00:00.000000000,4.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,其他
1796,182612,生理男,2012-05-17 00:00:00.000000000,475.0,13.0,國語文教材（六年級）學習扶助教材,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,國文
1797,182612,生理男,2012-05-17 00:00:00.000000000,1263.0,26.0,(112學年) 真平版─國小閩語【六下】,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,閩語
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21533,185418,生理男,2011-12-06 00:00:00.000000000,41789.0,2213.0,忠於朗讀,0.0,0.0,0.0,0.0,2213.0,2213.0,0.0,0.0,6,其他
21534,185419,生理男,2014-04-22 00:00:00.000000000,1102.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,其他
21535,185420,生理女,2013-12-13 00:00:00.000000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,其他
21536,185428,生理女,2016-09-06 00:00:00.000000000,635.0,50.0,國語文教材（一年級）學習扶助教材,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,國文


In [ ]:
# prompt: 將data以courseCategory分類後，將相同年級、同stuid的資料的本月瀏覽影片時間、本月測驗時間等欄位加總，保留其餘欄位，存到新的變數中

new_data = data.groupby(['courseCategory', 'grade', 'stuid'])[['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']].sum().reset_index()

# 保留其他欄位
for col in data.columns:
  if col not in ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間', 'courseCategory', 'grade', 'stuid']:
    new_data[col] = data.groupby(['courseCategory', 'grade', 'stuid'])[col].first().values
new_data = new_data[new_data['grade'] != 0]
new_data.head()

,courseCategory,grade,stuid,瀏覽影片時間,測驗時間,語音作業時間,音檔時間,gender,birth,在線時間,素材總使用時間,課程名稱,影片加總時間,測驗加總時間,語音作業加總時間,音檔加總時間
0,其他,1,182785,0.0,0.0,0.0,0.0,生理女,2017-05-10 00:00:00.000000000,2711.0,0.0,None,0.0,0.0,0.0,0.0
1,其他,1,184165,0.0,0.0,0.0,0.0,生理男,2017-05-29 00:00:00.000000000,123.0,0.0,None,0.0,0.0,0.0,0.0
2,其他,1,184404,0.0,0.0,0.0,0.0,生理男,2017-07-25 00:00:00.000000000,80.0,0.0,None,0.0,0.0,0.0,0.0
3,其他,1,185119,0.0,0.0,0.0,0.0,生理男,2017-01-07 00:00:00.000000000,3.0,0.0,None,0.0,0.0,0.0,0.0
4,其他,1,185122,0.0,0.0,0.0,0.0,生理女,2016-12-02 00:00:00.000000000,4274.0,0.0,None,0.0,0.0,0.0,0.0


## （一）總體學生學習分析

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Step 1: Create a pivot table with student IDs as rows and course names as columns
# Using '素材總使用時間' as the values column
pivot_subject_behavior = new_data.pivot_table(
    index='stuid',
    columns='課程名稱',
    values='素材總使用時間',
    fill_value=0
)

# Step 2: Split the courses into 3 parts (if the number of courses is divisible by 3)
num_courses = len(pivot_subject_behavior.columns)
part_size = num_courses // 3  # Divide the courses into 3 parts

# Calculate the correct number of parts (if there are more than 3 parts)
if num_courses % 3 != 0:
    course_parts = [
        pivot_subject_behavior.columns[i:i + part_size] for i in range(0, num_courses - part_size, part_size)
    ]
    # Ensure the last part contains the remaining courses
    course_parts.append(pivot_subject_behavior.columns[num_courses - (num_courses % 3):])
else:
    course_parts = [pivot_subject_behavior.columns[i:i + part_size] for i in range(0, num_courses, part_size)]

# Ensure there are exactly 3 parts
course_parts = course_parts[:3]

# Create a list to store the figures
figures = []

# Step 3: Create a heatmap for each part
for i, course_part in enumerate(course_parts):
    fig = go.Figure(data=go.Heatmap(
        z=pivot_subject_behavior[course_part].values,  # Heatmap data
        x=course_part,  # Course names on the x-axis
        y=pivot_subject_behavior.index,  # Student IDs on the y-axis
        colorscale='YlGnBu',  # Color scale
        zmin=0,  # Set the color scale's minimum value
        zmax=10000  # Set the color scale's maximum value
    ))

    # Step 4: Update layout for each figure
    fig.update_layout(
        title={
            "text": f"學生在各課程的素材總使用時間熱力圖 (2024年2月～2024年6月) - Part {i+1}",  # Title text
            "font": {"size": 24}  # Title font size
        },
        xaxis_title="課程",  # X-axis label
        yaxis_title="學生ID",  # Y-axis label
        height=800,  # Set chart height
        width=1200,  # Set chart width to 1200
        xaxis=dict(
            tickangle=90,  # Rotate x-axis labels by 90 degrees
            tickmode='array',  # Force all labels to display (avoid truncation)
            tickvals=[i for i in range(len(course_part))],  # Ensure all labels are shown
        ),
        yaxis=dict(
            tickmode='array',  # Optionally, adjust y-axis labels if needed
        )
    )

    # Add the figure to the list
    figures.append(fig)

# Step 5: Display all figures one by one
for fig in figures:
    fig.show()


print(f"「做伙來」讀課文素材總使用時間：",new_data[new_data['課程名稱'] == '「做伙來」讀課文  ']['素材總使用時間'].sum())
print(f"忠於朗讀素材總使用時間：",new_data[new_data['課程名稱'] == '忠於朗讀']['素材總使用時間'].sum())


「做伙來」讀課文素材總使用時間： 627903.0
忠於朗讀素材總使用時間： 111619.0


In [ ]:
import pandas as pd
import plotly.express as px

# 假設 new_data 已經包含所需的欄位，並且有 '課程名稱' 欄位來區分不同課程

# Step 1: Filter data for '「做伙來」讀課文' and '忠於朗讀'
course1_data = new_data[new_data['課程名稱'] == '「做伙來」讀課文  ']
course2_data = new_data[new_data['課程名稱'] == '忠於朗讀']

# Step 2: Melt the data to have the time columns ('瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間') as a single column
# and the usage time as another column
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Reshape course1_data and course2_data
course1_melted = pd.melt(course1_data, id_vars=['grade', 'stuid', '課程名稱'], value_vars=time_columns,
                         var_name='素材類型', value_name='使用時間(秒)')
course2_melted = pd.melt(course2_data, id_vars=['grade', 'stuid', '課程名稱'], value_vars=time_columns,
                         var_name='素材類型', value_name='使用時間(秒)')

# Step 3: Plot the box plots

# Box plot for '「做伙來」讀課文'
fig1 = px.box(
    course1_melted,
    x='素材類型',  # X軸為素材類型
    y='使用時間(秒)',  # Y軸為使用時間
    points='all',  # 顯示所有點
    color='素材類型',  # 根據素材類型分色
    title='「做伙來」讀課文 素材總使用時間(2024年2月～2024年6月)',
    labels={'素材類型': '素材類型', '使用時間(秒)': '素材總使用時間(秒)'}
)

# Box plot for '忠於朗讀'
fig2 = px.box(
    course2_melted,
    x='素材類型',  # X軸為素材類型
    y='使用時間(秒)',  # Y軸為使用時間
    points='all',  # 顯示所有點
    color='素材類型',  # 根據素材類型分色
    title='忠於朗讀 素材總使用時間(2024年2月～2024年6月)',
    labels={'素材類型': '素材類型', '使用時間(秒)': '素材總使用時間(秒)'}
)
fig1.update_layout(
    title_font=dict(size=24)  # 設置標題字體大小為 24
)
fig2.update_layout(
    title_font=dict(size=24)  # 設置標題字體大小為 24
)
# 自定義點的樣式
fig1.update_traces(marker=dict(size=0.5))  # 設定點的大小
fig2.update_traces(marker=dict(size=0.5))  # 設定點的大小
# Display the plots
fig1.show()
fig2.show()
fig1.write_html("1-2-1.html")
fig2.write_html("1-2-2.html")



In [ ]:
import pandas as pd

# Step 1: Filter data for '「做伙來」讀課文' and '忠於朗讀'
course1_data = new_data[new_data['課程名稱'] == '「做伙來」讀課文  ']
course2_data = new_data[new_data['課程名稱'] == '忠於朗讀']

# Step 2: Melt the data to have the time columns ('瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間') as a single column
# and the usage time as another column
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Reshape course1_data and course2_data
course1_melted = pd.melt(course1_data, id_vars=['grade', 'stuid', '課程名稱'], value_vars=time_columns,
                         var_name='素材類型', value_name='使用時間(秒)')
course2_melted = pd.melt(course2_data, id_vars=['grade', 'stuid', '課程名稱'], value_vars=time_columns,
                         var_name='素材類型', value_name='使用時間(秒)')

# Combine the two reshaped dataframes into one
combined_data = pd.concat([course1_melted, course2_melted])

# Create a summary table for the combined data
summary_table = combined_data.groupby(['課程名稱', '素材類型']).agg({
    '使用時間(秒)': ['mean', 'std', 'min', 'max', 'median', 'count']
}).reset_index()

# Flatten the column names (to remove multi-level column index)
summary_table.columns = ['課程名稱', '素材類型', '平均使用時間(秒)', '使用時間標準差(秒)', '最小使用時間(秒)', '最大使用時間(秒)', '中位數使用時間(秒)', '樣本數']

# Create Markdown table from the summary_table DataFrame
markdown_table = '| 課程名稱 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |\n'
markdown_table += '|---|---|---|---|---|---|---|---|\n'

for _, row in summary_table.iterrows():
    markdown_table += f"| {row['課程名稱']} | {row['素材類型']} | {row['平均使用時間(秒)']:.2f} | {row['使用時間標準差(秒)']:.2f} | {row['最小使用時間(秒)']:.2f} | {row['最大使用時間(秒)']:.2f} | {row['中位數使用時間(秒)']:.2f} | {row['樣本數']} |\n"

# Print the markdown table
print(markdown_table)

| 課程名稱 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |
|---|---|---|---|---|---|---|---|
| 「做伙來」讀課文   | 測驗時間 | 0.71 | 12.80 | 0.00 | 236.00 | 0.00 | 648 |
| 「做伙來」讀課文   | 瀏覽影片時間 | 571.00 | 829.93 | 0.00 | 4011.00 | 205.50 | 648 |
| 「做伙來」讀課文   | 語音作業時間 | 129.22 | 270.15 | 0.00 | 2750.00 | 35.00 | 648 |
| 「做伙來」讀課文   | 音檔時間 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 648 |
| 忠於朗讀 | 測驗時間 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 21 |
| 忠於朗讀 | 瀏覽影片時間 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 21 |
| 忠於朗讀 | 語音作業時間 | 8156.81 | 3191.83 | 2701.00 | 13293.00 | 7088.00 | 21 |
| 忠於朗讀 | 音檔時間 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 21 |



## （二）性別年級學習差異分析

In [ ]:
import pandas as pd
import plotly.express as px

# 假設 new_data 已經包含所需的欄位，並且有 'grade' 欄位來區分年級
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Step 1: 將資料 Reshape 為長格式
# Melt new_data，使 time_columns 成為一列
melted_data = pd.melt(
    new_data,
    id_vars=['grade'],  # 固定的識別欄位，移除 'gender'
    value_vars=time_columns,  # 要展開的欄位
    var_name='素材類型',  # 新欄位名稱：素材類型
    value_name='使用時間(秒)'  # 新欄位名稱：使用時間
)

# Step 2: 繪製不同素材類型在1-6年級的使用時間
fig = px.box(
    melted_data,
    x='素材類型',  # X軸為素材類型
    y='使用時間(秒)',  # Y軸為使用時間
    color='grade',  # 根據年級分色
    points='all',  # 顯示所有點
    title='1-6年級各素材類型的使用時間 (2024年2月～2024年6月)',
    labels={'grade': '年級', '使用時間(秒)': '素材使用時間(秒)', '素材類型': '素材類型'}
)

# 自定義點的樣式
fig.update_traces(marker=dict(size=0.5))  # 設定點的大小

# 更新 Y 軸範圍
fig.update_layout(
    yaxis=dict(
        range=[-100, 3000]  # 設定 Y 軸的最小值和最大值
    ),
    title_font=dict(size=24), # 設置標題字體大小為 24
    xaxis_title='素材類型',
    yaxis_title='素材使用時間(秒)'
)

# 顯示圖表
fig.show()
fig.write_html("2-1-4.html")

In [ ]:
import pandas as pd

# 假設 new_data 已經是原始資料
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Step 1: 將資料 Reshape 為長格式
melted_data = pd.melt(
    new_data,
    id_vars=['grade'],  # 確保 'grade' 在原始資料中存在
    value_vars=time_columns,  # 要展開的欄位
    var_name='素材類型',  # 新欄位名稱：素材類型
    value_name='使用時間(秒)'  # 新欄位名稱：使用時間
)

# Step 2: 計算每個年級和素材類型的多個統計量
summary_data = melted_data.groupby(['grade', '素材類型'])['使用時間(秒)'].agg(
    average='mean',
    std_dev='std',
    min='min',
    max='max',
    median='median',
    count='count'
).reset_index()

# Step 3: 生成 Markdown 表格
markdown_table = '| 年級 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |\n'
markdown_table += '|---|---|---|---|---|---|---|---|\n'

# Step 4: 將結果加入到表格中
for _, row in summary_data.iterrows():
    markdown_table += f"| {row['grade']} | {row['素材類型']} | {row['average']:.2f} | {row['std_dev']:.2f} | {row['min']:.2f} | {row['max']:.2f} | {row['median']:.2f} | {row['count']} |\n"

# 顯示 Markdown 表格
print(markdown_table)



| 年級 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |
|---|---|---|---|---|---|---|---|
| 1 | 測驗時間 | 5.72 | 34.33 | 0.00 | 206.00 | 0.00 | 36 |
| 1 | 瀏覽影片時間 | 74.72 | 172.95 | 0.00 | 739.00 | 0.00 | 36 |
| 1 | 語音作業時間 | 42.81 | 105.87 | 0.00 | 501.00 | 0.00 | 36 |
| 1 | 音檔時間 | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 | 36 |
| 2 | 測驗時間 | 157.23 | 389.36 | 0.00 | 4381.00 | 0.00 | 303 |
| 2 | 瀏覽影片時間 | 88.29 | 210.19 | 0.00 | 1837.00 | 0.00 | 303 |
| 2 | 語音作業時間 | 428.60 | 1840.98 | 0.00 | 21593.00 | 0.00 | 303 |
| 2 | 音檔時間 | 1.10 | 11.06 | 0.00 | 127.00 | 0.00 | 303 |
| 3 | 測驗時間 | 72.04 | 378.99 | 0.00 | 6640.00 | 0.00 | 918 |
| 3 | 瀏覽影片時間 | 159.82 | 906.73 | 0.00 | 22929.00 | 0.00 | 918 |
| 3 | 語音作業時間 | 204.53 | 695.30 | 0.00 | 5797.00 | 0.00 | 918 |
| 3 | 音檔時間 | 17.52 | 114.71 | 0.00 | 1201.00 | 0.00 | 918 |
| 4 | 測驗時間 | 37.62 | 156.42 | 0.00 | 2155.00 | 0.00 | 791 |
| 4 | 瀏覽影片時間 | 442.04 | 1281.85 | 0.00 | 12526.00 | 0.00 | 791 |
| 4 | 語音作業時間 | 163.13 | 443.65 | 0.00 

In [ ]:
import pandas as pd
import plotly.express as px

# Assume new_data contains necessary columns, including 'gender' for physiological sex classification
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Step 1: Reshape the data into long format
# Melt new_data to make time_columns into a single column
melted_data = pd.melt(
    new_data,
    id_vars=['gender'],  # Keep 'gender' as the identifier
    value_vars=time_columns,  # Columns to be unpivoted into a single column
    var_name='素材類型',  # Name for the new column representing material types
    value_name='使用時間(秒)'  # Name for the new column representing usage time
)

category_order = {"gender": ["生理男", "生理女"]}

# Step 2: Create a box plot to visualize usage time by gender for each material type
fig = px.box(
    melted_data,
    x='素材類型',  # X-axis: Material types
    y='使用時間(秒)',  # Y-axis: Usage time
    color='gender',  # Color by gender
    points='all',  # Display all individual data points
    title='各素材類型在生理性別下的使用時間 (2024年2月～2024年6月)',
    labels={
        'gender': '生理性別',
        '使用時間(秒)': '素材使用時間(秒)',
        '素材類型': '素材類型'
    },
    color_discrete_map={'生理男': 'blue', '生理女': 'red'},  # Custom color for male and female
    category_orders=category_order
)

# Customize marker size for individual points
fig.update_traces(marker=dict(size=0.5))  # Adjust the size of points

# Update Y-axis range for better visualization
fig.update_layout(
    yaxis=dict(
        range=[-100, 3000]  # Set Y-axis range to cover a good range of values
    ),
    title_font=dict(size=24),  # Set title font size
    xaxis_title='素材類型',  # X-axis title
    yaxis_title='素材使用時間(秒)'  # Y-axis title
)

# Show the plot
fig.show()
fig.write_html("2-1-3.html")

In [ ]:
import pandas as pd
import plotly.express as px

# Assume new_data contains necessary columns, including 'gender' for physiological sex classification
time_columns = ['瀏覽影片時間', '測驗時間', '語音作業時間', '音檔時間']

# Step 1: Reshape the data into long format
# Melt new_data to make time_columns into a single column
melted_data = pd.melt(
    new_data,
    id_vars=['gender'],  # Keep 'gender' as the identifier
    value_vars=time_columns,  # Columns to be unpivoted into a single column
    var_name='素材類型',  # Name for the new column representing material types
    value_name='使用時間(秒)'  # Name for the new column representing usage time
)

# Step 2: Calculate statistics for each group (gender and material type)
summary_data = melted_data.groupby(['gender', '素材類型'])['使用時間(秒)'].agg(
    average='mean',  # Mean usage time
    std_dev='std',  # Standard deviation of usage time
    min_time='min',  # Minimum usage time
    max_time='max',  # Maximum usage time
    median_time='median',  # Median usage time
    count='count'  # Sample count
).reset_index()

# Step 3: Create the Markdown table for HackMD
markdown_table = '| 生理性別 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |\n'
markdown_table += '|---|---|---|---|---|---|---|---|\n'

# Iterate over the rows to create the table content
for _, row in summary_data.iterrows():
    markdown_table += f"| {row['gender']} | {row['素材類型']} | {row['average']:.2f} | {row['std_dev']:.2f} | {row['min_time']:.2f} | {row['max_time']:.2f} | {row['median_time']:.2f} | {row['count']} |\n"

# Print the Markdown table so it can be copied into HackMD
print(markdown_table)






| 生理性別 | 素材類型 | 平均使用時間(秒) | 使用時間標準差(秒) | 最小使用時間(秒) | 最大使用時間(秒) | 中位數使用時間(秒) | 樣本數 |
|---|---|---|---|---|---|---|---|
| 生理女 | 測驗時間 | 449.56 | 3379.17 | 0.00 | 57292.00 | 0.00 | 1994 |
| 生理女 | 瀏覽影片時間 | 290.41 | 1442.60 | 0.00 | 46272.00 | 0.00 | 1994 |
| 生理女 | 語音作業時間 | 360.57 | 1537.17 | 0.00 | 21593.00 | 0.00 | 1994 |
| 生理女 | 音檔時間 | 6.18 | 59.38 | 0.00 | 1201.00 | 0.00 | 1994 |
| 生理男 | 測驗時間 | 324.24 | 2915.46 | 0.00 | 76256.00 | 0.00 | 2048 |
| 生理男 | 瀏覽影片時間 | 277.31 | 873.30 | 0.00 | 10372.00 | 0.00 | 2048 |
| 生理男 | 語音作業時間 | 272.04 | 1097.60 | 0.00 | 17675.00 | 0.00 | 2048 |
| 生理男 | 音檔時間 | 5.83 | 63.85 | 0.00 | 1108.00 | 0.00 | 2048 |



## （三）不同年級 PCA + KMeans + OneHotEncoding 分析

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

grade_behavior_summary = new_data.groupby("stuid").agg({
    '素材總使用時間': 'sum',
    '瀏覽影片時間': 'sum',
    '測驗時間': 'sum',
    '語音作業時間': 'sum',
    '音檔時間': 'sum',
    'grade': 'first'
})
grade_behavior_summary.reset_index(inplace=True)
grade_behavior_summary.head()

,stuid,素材總使用時間,瀏覽影片時間,測驗時間,語音作業時間,音檔時間,grade
0,116930,516.0,511.0,0.0,0.0,0.0,6
1,116939,0.0,0.0,0.0,0.0,0.0,6
2,116947,0.0,0.0,0.0,0.0,0.0,6
3,116957,3009.0,2280.0,0.0,0.0,0.0,6
4,127462,425.0,0.0,0.0,688.0,0.0,6


In [ ]:
# 定義對類別資料進行 One-Hot Encoding 的轉換器
encoder = OneHotEncoder(sparse_output=False)
encoded_grade = encoder.fit_transform(grade_behavior_summary[['grade']])
encoded_grade_df = pd.DataFrame(encoded_grade, columns=encoder.get_feature_names_out(['grade']))
# grade_behavior_summary = grade_behavior_summary.drop(columns=['grade'])
grade_behavior_summary = pd.concat([grade_behavior_summary, encoded_grade_df], axis=1)

kmeans_columns = ['素材總使用時間'] + [col for col in grade_behavior_summary.columns if col.startswith('grade_')]

pca = PCA(n_components=2)
components = pca.fit_transform(grade_behavior_summary[kmeans_columns])
grade_behavior_summary['pca1'] = components[:, 0]
grade_behavior_summary['pca2'] = components[:, 1]

kmeans = KMeans(n_clusters=3)
grade_behavior_summary['cluster'] = kmeans.fit_predict(grade_behavior_summary[kmeans_columns])

fig = px.scatter(
    grade_behavior_summary,
    x='pca1',
    y='pca2',
    color='cluster',
    title="不同年級的學生行為聚集結果 (2024年2月～2024年6月)",
    labels={'pca1': '主成分1', 'pca2': '主成分2', 'cluster': '聚類群組'},
    hover_data=kmeans_columns  # 在互動中顯示更多細節
)

# 更新圖表的佈局
fig.update_layout(
    xaxis_title="主成分1",
    yaxis_title="主成分2",
    legend_title="群組",
    height=600,
    width=800,
    title_font=dict(size=24) # 設置標題字體大小為 24

)

# 顯示圖表
fig.show()
fig.write_html("3-1-1.html")

根據手肘法則 (Elbow Method)，我們應該選擇折彎點 (elbow point)，即 SSE（誤差平方和）開始減少趨勢變緩的位置。從圖中來看，折彎點大約是在3群，因為從 3 開始，SSE 的下降幅度顯著變小。

In [ ]:
sse = []  # To store sum of squared errors for each K value
k_range = range(1, 11)  # Test for 1 to 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(grade_behavior_summary[['pca1', 'pca2']])  # Fit only on PCA components
    sse.append(kmeans.inertia_)  # SSE is stored in inertia_

# Step 7: Plot the Elbow Method using Plotly Express
elbow_df = pd.DataFrame({'Clusters': k_range, 'SSE': sse})

fig_elbow = px.line(elbow_df, x='Clusters', y='SSE',
                    title='手肘法則判斷最佳群集數量 (不同年級的學生行為聚集)',
                    labels={'Clusters': '群集數量', 'SSE': '平方誤差總和'},
                    markers=True)
fig_elbow.update_layout(
    title_font=dict(size=24) # 設置標題字體大小為 24
)
fig_elbow.update_traces(line=dict(color='blue', width=3), marker=dict(size=8))
fig_elbow.show()

In [ ]:
grade_behavior_summary_sorted = grade_behavior_summary.sort_values(by=['grade','cluster'], ascending=[True, True])

fig_box = px.box(
    grade_behavior_summary_sorted,
    x='cluster',  # Cluster
    y='素材總使用時間',  # PCA1 as usage proxy (you can also use PCA2 or 'Total Usage' if available)
    color='grade',  # Group by grade
    points='all',  # Show all individual points
    title='每個群集與年級的素材總使用時間分佈 (2024年2月～2024年6月)',
    labels={'cluster': '群集', '素材總使用時間': '素材總使用時間（秒）', 'grade': '年級'},
    color_discrete_map={
        1: 'red',    # 1年級 -> Red
        2: 'green',  # 2年級 -> Green
        3: 'blue',   # 3年級 -> Blue
        4: 'orange', # 4年級 -> Orange
        5: 'purple', # 5年級 -> Purple
        6: 'yellow'  # 6年級 -> Yellow
    }
)
fig_box.update_layout(
    title_font=dict(size=24) # 設置標題字體大小為 24
)

# Customize the appearance of the box plot
fig_box.update_traces(marker=dict(opacity=0))  # Customize marker size

# Show the box plot
fig_box.show()
fig_box.write_html("3-1-2.html")


In [ ]:
import pandas as pd

# 假設 grade_behavior_summary 是你的資料框
# 創建一個 Markdown 表格的表頭
markdown_table = '| 群集 | 年級 | 最小使用時間(秒) | Q1使用時間(秒) | 中位數使用時間(秒) | Q3使用時間(秒) | 最大使用時間(秒) |\n'
markdown_table += '|---|---|---|---|---|---|---|\n'

# 確保群集是排序為 0, 1, 2 且年級是從 1 到 6 排序
for cluster_value in sorted(grade_behavior_summary['cluster'].unique()):
    for grade_value in sorted(grade_behavior_summary['grade'].unique()):
        # 篩選資料
        filtered_data = grade_behavior_summary[
            (grade_behavior_summary['cluster'] == cluster_value) &
            (grade_behavior_summary['grade'] == grade_value)
        ]['素材總使用時間']

        # 計算統計數據
        min_val = filtered_data.min()
        q1_val = filtered_data.quantile(0.25)
        median_val = filtered_data.median()
        q3_val = filtered_data.quantile(0.75)
        max_val = filtered_data.max()

        # 添加統計數據到 Markdown 表格
        markdown_table += f"| {cluster_value} | {grade_value} | {min_val:.2f} | {q1_val:.2f} | {median_val:.2f} | {q3_val:.2f} | {max_val:.2f} |\n"

# 打印 Markdown 表格
print(markdown_table)


| 群集 | 年級 | 最小使用時間(秒) | Q1使用時間(秒) | 中位數使用時間(秒) | Q3使用時間(秒) | 最大使用時間(秒) |
|---|---|---|---|---|---|---|
| 0 | 1 | nan | nan | nan | nan | nan |
| 0 | 2 | 4186.00 | 4549.00 | 4912.00 | 6015.50 | 7119.00 |
| 0 | 3 | 3860.00 | 5159.00 | 7385.00 | 11060.00 | 14548.00 |
| 0 | 4 | 3775.00 | 4013.50 | 4250.50 | 5042.50 | 7868.00 |
| 0 | 5 | 3813.00 | 5152.75 | 7030.50 | 10121.75 | 14615.00 |
| 0 | 6 | 3795.00 | 4876.25 | 6003.00 | 8384.25 | 13841.00 |
| 1 | 1 | nan | nan | nan | nan | nan |
| 1 | 2 | 19940.00 | 22350.50 | 27296.50 | 30805.25 | 43584.00 |
| 1 | 3 | 15943.00 | 16540.00 | 21303.00 | 22166.00 | 22666.00 |
| 1 | 4 | nan | nan | nan | nan | nan |
| 1 | 5 | 16014.00 | 16267.00 | 16965.00 | 20690.00 | 23892.00 |
| 1 | 6 | 14992.00 | 16713.00 | 18434.00 | 20636.50 | 22839.00 |
| 2 | 1 | 0.00 | 0.00 | 0.00 | 109.00 | 1166.00 |
| 2 | 2 | 0.00 | 36.50 | 282.00 | 569.25 | 3072.00 |
| 2 | 3 | 0.00 | 0.00 | 136.50 | 645.00 | 3388.00 |
| 2 | 4 | 0.00 | 15.00 | 268.50 | 741.25 | 3613.00 |
| 2 

In [ ]:
# Printing header for HackMD table format
print("\n每個群集中至少有兩個年級共同參與的課程 (按參與年級分組):")
print("| 群集 | 參與年級 | 共同參與的課程 |")
print("|------|----------|----------------|")

# Loop through each cluster
for cluster in sorted(grade_behavior_summary['cluster'].unique()):
    cluster_students = grade_behavior_summary[grade_behavior_summary['cluster'] == cluster]
    grades_in_cluster = sorted(cluster_students['grade'].unique())

    grade_courses_dict = {}

    for grade in grades_in_cluster:
        grade_students = cluster_students[cluster_students['grade'] == grade]
        grade_student_ids = grade_students['stuid']
        grade_courses = data[data['stuid'].isin(grade_student_ids)]['課程名稱'].unique()
        grade_courses_dict[grade] = set(grade_courses)

    shared_courses_by_grades = {}
    all_courses = set.union(*grade_courses_dict.values()) if grade_courses_dict else set()

    for course in all_courses:
        participating_grades = [grade for grade, courses in grade_courses_dict.items() if course in courses]
        if len(participating_grades) >= 2:
            grades_tuple = tuple(sorted(participating_grades))
            if grades_tuple not in shared_courses_by_grades:
                shared_courses_by_grades[grades_tuple] = []
            shared_courses_by_grades[grades_tuple].append(course)

    if shared_courses_by_grades:
        for grades, courses in shared_courses_by_grades.items():
            grades_str = ", ".join(map(str, grades))
            # Ensure all courses are converted to string before joining
            courses_str = ", ".join(map(str, courses))
            print(f"| 群集{cluster} | {grades_str} | {courses_str} |")
    else:
        print(f"群集{cluster} 沒有課程有兩個以上年級共同參與。")

print()  # Add a blank line at the end




每個群集中至少有兩個年級共同參與的課程 (按參與年級分組):
| 群集 | 參與年級 | 共同參與的課程 |
|------|----------|----------------|
| 群集0 | 3, 4, 5, 6 | nan, 「做伙來」讀課文  , 學習吧－節慶英文故事館  |
| 群集0 | 2, 3, 5 | 唐詩精選（五言、七言絕句） |
| 群集0 | 4, 5 | 國語日報 - 讀報課程【週一~週五更新】（文章授權：國語日報社） |
| 群集0 | 4, 6 | Where are you?, 112下 Where are you going? |
| 群集0 | 5, 6 | 閱讀寫作齊步走【小壁虎老師】 |
| 群集0 | 3, 5 | AI程式課程（創造栗）, 成語故事 - 典故由來3 |
| 群集0 | 3, 6 | 113 reading |
| 群集1 | 3, 5 | nan |
| 群集1 | 2, 5 | 成語故事 - 從典故到生活 |
| 群集2 | 1, 3, 4, 5 | 水土保持好好玩 |
| 群集2 | 3, 5 | 國語日報 - 讀報課程【週一~週五更新】（文章授權：國語日報社）, (112學年) 真平版─國小閩語【三下】, 🏆112下學期-新北市國小【中年級】學習扶助自我挑戰超能賽 |
| 群集2 | 3, 4, 5, 6 | 節慶英語故事－聽故事學英語, 唐詩精選（五言、七言絕句） |
| 群集2 | 4, 5, 6 | AI程式課程（創造栗）, 2024線上寒假作業（4-6年級）, 英語初級閱讀【博幼基金會授權】, 水保小教室- 呼叫水保隊長 |
| 群集2 | 5, 6 | 112下 Where are you going?, 112下503國語, 112-2 Fish and Chips 炸魚薯條, 🏆112下學期-新北市國小【高年級】學習扶助自我挑戰超能賽, Where are you?, G5 篩選測驗練習(2023), 憲明國小 校歌 |
| 群集2 | 1, 2, 3, 4, 5, 6 | 「做伙來」讀課文  , 成語故事 - 從典故到生活, nan |
| 群集2 | 2, 3, 4, 5, 6 | 動物星球頻道－動物檢定, 這週世界發生了什麼？（TVBS 合作課程）, 唐詩朗誦大挑戰, 學習吧

## （四）不同性別 PCA + KMeans + OneHotEncoding 分析

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

gender_behavior_summary = new_data.groupby("stuid").agg({
    '素材總使用時間': 'sum',
    '瀏覽影片時間': 'sum',
    '測驗時間': 'sum',
    '語音作業時間': 'sum',
    '音檔時間': 'sum',
    'gender': 'first'
})
gender_behavior_summary.reset_index(inplace=True)
gender_behavior_summary.head()

,stuid,素材總使用時間,瀏覽影片時間,測驗時間,語音作業時間,音檔時間,gender
0,116930,516.0,511.0,0.0,0.0,0.0,生理男
1,116939,0.0,0.0,0.0,0.0,0.0,生理男
2,116947,0.0,0.0,0.0,0.0,0.0,生理女
3,116957,3009.0,2280.0,0.0,0.0,0.0,生理女
4,127462,425.0,0.0,0.0,688.0,0.0,生理女


In [ ]:
# 定義對類別資料進行 One-Hot Encoding 的轉換器
encoder = OneHotEncoder(sparse_output=False)
encoded_gender = encoder.fit_transform(gender_behavior_summary[['gender']])
encoded_gender_df = pd.DataFrame(encoded_gender, columns=encoder.get_feature_names_out(['gender']))
# gender_behavior_summary = gender_behavior_summary.drop(columns=['gender'])
gender_behavior_summary = pd.concat([gender_behavior_summary, encoded_gender_df], axis=1)

kmeans_columns = ['素材總使用時間'] + [col for col in gender_behavior_summary if col.startswith('gender_')]

pca = PCA(n_components=2)
components = pca.fit_transform(gender_behavior_summary[kmeans_columns])
gender_behavior_summary['pca1'] = components[:, 0]
gender_behavior_summary['pca2'] = components[:, 1]

kmeans = KMeans(n_clusters=3)
gender_behavior_summary['cluster'] = kmeans.fit_predict(gender_behavior_summary[kmeans_columns])

fig = px.scatter(
    gender_behavior_summary,
    x='pca1',
    y='pca2',
    color='cluster',
    title="不同性別的學生行為聚類結果 (2024年2月～2024年6月)",
    labels={'pca1': '主成分1', 'pca2': '主成分2', 'cluster': '聚類群組'},
    hover_data=kmeans_columns  # 在互動中顯示更多細節
)

# 更新圖表的佈局
fig.update_layout(
    xaxis_title="主成分1",
    yaxis_title="主成分2",
    legend_title="群組",
    height=600,
    width=800,
    title_font=dict(size=24) # 設置標題字體大小為 24
)

# 顯示圖表
fig.show()
fig.write_html("4-1-1.html")

根據手肘法則 (Elbow Method)，我們應該選擇折彎點 (elbow point)，即 SSE（誤差平方和）開始減少趨勢變緩的位置。從圖中來看，折彎點大約是在3群，因為從 3 開始，SSE 的下降幅度顯著變小。

In [ ]:
sse = []  # To store sum of squared errors for each K value
k_range = range(1, 11)  # Test for 1 to 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(gender_behavior_summary[['pca1', 'pca2']])  # Fit only on PCA components
    sse.append(kmeans.inertia_)  # SSE is stored in inertia_

# Step 7: Plot the Elbow Method using Plotly Express
elbow_df = pd.DataFrame({'Clusters': k_range, 'SSE': sse})

fig_elbow = px.line(elbow_df, x='Clusters', y='SSE',
                    title='手肘法則判斷最佳群集數量 (不同性別的學生行為聚集)',
                    labels={'Clusters': '群集數量', 'SSE': '平方誤差總和'},
                    markers=True)
fig_elbow.update_layout(
    title_font=dict(size=24) # 設置標題字體大小為 24
)
fig_elbow.update_traces(line=dict(color='blue', width=3), marker=dict(size=8))
fig_elbow.show()

In [ ]:
fig_box = px.box(
    gender_behavior_summary,
    x='cluster',  # Cluster
    y='素材總使用時間',  # PCA1 as usage proxy (you can also use PCA2 or 'Total Usage' if available)
    color='gender',  # Group by grade
    points='all',  # Show all individual points
    title='每個群集與性別的總使用時間分佈 (2024年2月～2024年6月)',
    labels={'cluster': '群集', '素材總使用時間': '素材總使用時間（秒）', 'gender': '生理性別'},
    color_discrete_map={
        '生理男': 'blue',    # 生理男 -> blue
        '生理女': 'red',  # 生理女 -> red
    }
)

# Customize the appearance of the box plot
fig_box.update_traces(marker=dict(opacity=0))  # Customize marker size

fig_box.update_layout(
    title_font=dict(size=24) # 設置標題字體大小為 24
)
# Show the box plot
fig_box.show()
fig_box.write_html("4-1-2.html")

In [ ]:
import pandas as pd

# Assuming gender_behavior_summary and data are already defined

# Create an empty list to store the rows for the markdown table
markdown_rows = []

# Iterate over each cluster
for cluster in sorted(gender_behavior_summary['cluster'].unique()):
    # Filter the data for the current cluster
    cluster_data = gender_behavior_summary[gender_behavior_summary['cluster'] == cluster]

    # Get the unique genders in the current cluster
    genders_in_cluster = cluster_data['gender'].unique()

    # Iterate over each gender in the current cluster
    for gender in genders_in_cluster:
        # Filter the data for the current gender
        gender_data = cluster_data[cluster_data['gender'] == gender]
        gender_student_ids = gender_data['stuid']

        # Get the total usage time for the students of the current gender
        gender_usage_time = data[data['stuid'].isin(gender_student_ids)]['素材總使用時間']

        # Calculate statistics for the total usage time
        if not gender_usage_time.empty:
            min_usage = gender_usage_time.min()
            max_usage = gender_usage_time.max()
            q1_usage = gender_usage_time.quantile(0.25)
            median_usage = gender_usage_time.median()
            q3_usage = gender_usage_time.quantile(0.75)
            total_usage = gender_usage_time.sum()

            # Add a row to the markdown table
            markdown_rows.append([
                cluster,
                gender,
                min_usage,
                q1_usage,
                median_usage,
                q3_usage,
                max_usage,
                total_usage
            ])

# Convert the rows to a DataFrame
markdown_df = pd.DataFrame(markdown_rows, columns=[
    '群集', '生理性別', '最小使用時間 (秒)', '第一四分位數 (秒)', '中位數 (秒)', '第三四分位數 (秒)', '最大使用時間 (秒)', '總使用時間 (秒)'
])

# Print the markdown table header
markdown_table = "| 群集 | 生理性別 | 最小使用時間 (秒) | 第一四分位數 (秒) | 中位數 (秒) | 第三四分位數 (秒) | 最大使用時間 (秒) | 總使用時間 (秒) |\n"
markdown_table += "|---|---|---|---|---|---|---|---|\n"

# Add rows to the table
for _, row in markdown_df.iterrows():
    markdown_table += f"| {row['群集']} | {row['生理性別']} | {row['最小使用時間 (秒)']} | {row['第一四分位數 (秒)']} | {row['中位數 (秒)']} | {row['第三四分位數 (秒)']} | {row['最大使用時間 (秒)']} | {row['總使用時間 (秒)']} |\n"

# Print the final markdown table
print(markdown_table)


| 群集 | 生理性別 | 最小使用時間 (秒) | 第一四分位數 (秒) | 中位數 (秒) | 第三四分位數 (秒) | 最大使用時間 (秒) | 總使用時間 (秒) |
|---|---|---|---|---|---|---|---|
| 0 | 生理女 | 0.0 | 1053.5 | 2549.0 | 4742.0 | 30556.0 | 1441157.0 |
| 0 | 生理男 | 0.0 | 986.0 | 2326.0 | 4116.0 | 24020.0 | 837624.0 |
| 1 | 生理男 | 0.0 | 0.0 | 128.0 | 602.0 | 57466.0 | 2030775.0 |
| 1 | 生理女 | 0.0 | 0.0 | 71.0 | 533.25 | 44677.0 | 1844454.0 |
| 2 | 生理女 | 0.0 | 2649.0 | 9624.0 | 19080.0 | 32389.0 | 746411.0 |
| 2 | 生理男 | 0.0 | 3482.75 | 10348.0 | 14856.75 | 27731.0 | 436273.0 |



In [ ]:
# Printing header for HackMD table format
print("\n每個群集的課程使用時間排行榜 (先共同，後按性別分開):")

# Loop through each cluster
for cluster in sorted(gender_behavior_summary['cluster'].unique()):
    cluster_students = gender_behavior_summary[gender_behavior_summary['cluster'] == cluster]

    # Find the genders in the cluster
    gender_in_cluster = cluster_students['gender'].unique()

    # Get the total usage time for all students in the cluster
    all_student_ids = cluster_students['stuid']
    all_courses = data[data['stuid'].isin(all_student_ids)][['素材總使用時間', '課程名稱']]

    # Summarize usage time for each course (all genders)
    all_course_usage = all_courses.groupby('課程名稱')['素材總使用時間'].sum()
    used_courses_all = all_course_usage[all_course_usage > 0]

    # Common courses (for all genders) leaderboard
    print(f"\n### 群集 {cluster} - 共同課程使用時間排行榜:")
    print("| 群集 | 課程名稱 | 參與性別 | 使用時間(秒數) | 排名 |")
    print("|------|----------|----------|----------------|------|")

    if not used_courses_all.empty:
        sorted_all_courses = used_courses_all.sort_values(ascending=False)
        top_all_courses = sorted_all_courses.head(3)
        bottom_all_courses = sorted_all_courses.tail(3)

        # Add "Top 3" and "Bottom 3" labels
        for course, usage_time in top_all_courses.items():
            print(f"| 群集{cluster} | {course} | 共同 | {usage_time:.2f} | 前三名 |")
        for course, usage_time in bottom_all_courses.items():
            print(f"| 群集{cluster} | {course} | 共同 | {usage_time:.2f} | 後三名 |")
    else:
        print(f"群集 {cluster}: 沒有共同參與的課程。")

    # Gender-specific course rankings
    for gender in gender_in_cluster:
        gender_students = cluster_students[cluster_students['gender'] == gender]
        gender_student_ids = gender_students['stuid']

        # Get total usage time for the gender-specific courses
        gender_courses = data[data['stuid'].isin(gender_student_ids)][['素材總使用時間', '課程名稱']]

        gender_course_usage = gender_courses.groupby('課程名稱')['素材總使用時間'].sum()
        used_courses = gender_course_usage[gender_course_usage > 0]

        print(f"\n### 群集 {cluster} - {gender} 性別課程使用時間排行榜:")
        print("| 群集 | 課程名稱 | 參與性別 | 使用時間(秒數) | 排名 |")
        print("|------|----------|----------|----------------|------|")

        if not used_courses.empty:
            sorted_courses = used_courses.sort_values(ascending=False)
            top_courses = sorted_courses.head(3)
            bottom_courses = sorted_courses.tail(3)

            # Add "Top 3" and "Bottom 3" labels for gender-specific rankings
            for course, usage_time in top_courses.items():
                print(f"| 群集{cluster} | {course} | {gender} | {usage_time:.2f} | 前三名 |")
            for course, usage_time in bottom_courses.items():
                print(f"| 群集{cluster} | {course} | {gender} | {usage_time:.2f} | 後三名 |")
        else:
            print(f"群集{cluster} - 性別 {gender}: 沒有學生參與課程。")

    print()  # Add a blank line after each cluster



每個群集的課程使用時間排行榜 (先共同，後按性別分開):

### 群集 0 - 共同課程使用時間排行榜:
| 群集 | 課程名稱 | 參與性別 | 使用時間(秒數) | 排名 |
|------|----------|----------|----------------|------|
| 群集0 | 「做伙來」讀課文   | 共同 | 269593.00 | 前三名 |
| 群集0 | 112憲明五忠(下) | 共同 | 262036.00 | 前三名 |
| 群集0 | 利澤118五孝下 | 共同 | 180419.00 | 前三名 |
| 群集0 | 玉田4下國語課文朗讀 | 共同 | 193.00 | 後三名 |
| 群集0 | 世界時事閱讀 | 共同 | 166.00 | 後三名 |
| 群集0 | 【兒童節活動】異口「童」聲 | 共同 | 145.00 | 後三名 |

### 群集 0 - 生理女 性別課程使用時間排行榜:
| 群集 | 課程名稱 | 參與性別 | 使用時間(秒數) | 排名 |
|------|----------|----------|----------------|------|
| 群集0 | 112憲明五忠(下) | 生理女 | 194440.00 | 前三名 |
| 群集0 | 利澤118五孝下 | 生理女 | 180419.00 | 前三名 |
| 群集0 | 「做伙來」讀課文   | 生理女 | 155495.00 | 前三名 |
| 群集0 | 世界時事閱讀 | 生理女 | 166.00 | 後三名 |
| 群集0 | 【兒童節活動】異口「童」聲 | 生理女 | 145.00 | 後三名 |
| 群集0 | 五下社會 | 生理女 | 87.00 | 後三名 |

### 群集 0 - 生理男 性別課程使用時間排行榜:
| 群集 | 課程名稱 | 參與性別 | 使用時間(秒數) | 排名 |
|------|----------|----------|----------------|------|
| 群集0 | 「做伙來」讀課文   | 生理男 | 114098.00 | 前三名 |
| 群集0 | 112憲明五忠(下) | 生理男 | 67596.00 | 前三名 |
| 群集0 | 112 學年 六年級下